In [2]:
import pandas as pd

# Load the JSON file
# df = pd.read_json('data/json_output/contractNLI_RG_baseline.json')
# df = pd.read_json('data/json_output/contractNLI_RG_zeroshot_CoT_improve1.json')
# df = pd.read_json('data/json_output/contractNLI_RG_zeroshot_CoT.json')
df = pd.read_json('../data/json_output/contractnli/gpt/query_answer_baseline_gpt4omini_k3.json')


In [3]:
print(df.columns)
# df.drop(columns=["thought"], inplace=True)
# print(df.columns)


Index(['user_input', 'response', 'retrieved_contexts'], dtype='object')


In [4]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas import evaluate
from datasets import Dataset    
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0.5))

dataset_ragas = Dataset.from_pandas(df)
print(dataset_ragas)
result = evaluate(
    dataset_ragas,
    metrics=[
        faithfulness,
        answer_relevancy
    ],
    llm = evaluator_llm
)

Dataset({
    features: ['user_input', 'response', 'retrieved_contexts'],
    num_rows: 194
})


Evaluating:   0%|          | 0/388 [00:00<?, ?it/s]

Exception raised in Job[151]: TimeoutError()


In [5]:
print(result)

{'faithfulness': 0.8279, 'answer_relevancy': 0.6179}


In [32]:
print(result)

{'faithfulness': 0.8104, 'answer_relevancy': 0.6758}


# Result

Baseline Prompt, eval: GPT 4o Mini temp 0.5
{'faithfulness': 0.8272, 'answer_relevancy': 0.5746}

CoT Prompt, eval: GPT 4o Mini temp 0.5
{'faithfulness': 0.4739, 'answer_relevancy': 0.6121}

CoT Improved Prompt, eval: GPT 4o Mini temp 0.5
{'faithfulness': 0.6213, 'answer_relevancy': 0.6459}

CoT Improved Prompt, eval: GPT 4o Mini temp 0.2
{'faithfulness': 0.6324, 'answer_relevancy': 0.6411}

Manually Written Prompt, eval: GPT 4o Mini temp 0.5
{'faithfulness': 0.8104, 'answer_relevancy': 0.6758}

In [16]:
# from ragas.dataset_schema import SingleTurnSample
# from ragas.metrics import Faithfulness
# from ragas.llms import LangchainLLMWrapper

# sample_index = 3

# sample = SingleTurnSample(
#     user_input=exp2_query_answer["user_input"][sample_index],
#     response=exp2_query_answer["response"][sample_index],
#     retrieved_contexts=exp2_query_answer["retrieved_contexts"][sample_index]
# )

# evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
# scorer = Faithfulness(llm=evaluator_llm)
# print(f"faithfulness: {await scorer.single_turn_ascore(sample)}")

# print(exp2_query_answer["user_input"][sample_index])
# print(exp2_query_answer["retrieved_contexts"][sample_index])
# print(exp2_query_answer["response"][sample_index])

In [3]:
print(df.columns)


Index(['user_input', 'response', 'retrieved_contexts'], dtype='object')


In [26]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import RougeScore

index = 25

sample = SingleTurnSample(
    response=df["response"][index],
    reference=" ".join(df["retrieved_contexts"][index])
)

scorer = RougeScore()
await scorer.single_turn_ascore(sample)

0.10471204188481675

In [27]:
from bert_score import score
P, R, F1 = score([df["response"][index]], [" ".join(df["retrieved_contexts"][index])], model_type="nlpaueb/legal-bert-base-uncased", num_layers=12)
print(F1.numpy())

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\w4rlo\Documents\Workspace\MSc\COMP0087\code\SNLP GENERATOR\response_gen_venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\w4rlo\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

AttributeError: BertTokenizerFast has no attribute max_len

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]